## Big Data Mining
### Project 2
### 2019-2020

Kyriakos   Zormpas  
Alexandros Zotos  
Theodora  Koutsothanasi  
Evangelia Santorinaiou  dsc19024  

## Libraries

In [ ]:
#!pip install namegenerator

In [1]:
import pandas as pd
import numpy as np

import namegenerator
from datetime import datetime, timedelta
import random
import time



### Dataset Creation

Our scenario in order to build our dataset is that we have all the restaurants of the world, name, id, when they were founded and their age, their coordinates, their speciality and their reviews. 

In [2]:
def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

def random_date(start, end, prop):
    return str_time_prop(start, end, '%m/%d/%Y', prop)

In [26]:
def get_latitude_longititude():
    df = pd.read_csv('World.csv')
    return df

In [30]:
df = get_latitude_longititude()[:100]
df

,Country,City,Latitude,Longitude
0,ad,aixas,42.483333,1.466667
1,ad,aixirivali,42.466667,1.500000
2,ad,andorra,42.500000,1.516667
3,ad,ansalonga,42.566667,1.516667
4,ad,anyos,42.533333,1.533333
5,ad,arans,42.583333,1.516667
6,ad,aubinya,42.450000,1.500000
7,ad,el pui,42.550000,1.516667
8,ad,el serrat,42.616667,1.550000
9,ad,engordany,42.516667,1.550000


In [19]:
restaurant_df = pd.DataFrame(columns=['id', 'name', 'cid', 'sid', 'rid', 'founded', 'age'])


for i in range(100):
    r_date = random_date("01/01/2008", "12/31/2019", random.random())
    
    restaurant_df.loc[i] = [np.random.randint(100000, 999999), 
                                   namegenerator.gen(), 
                                   np.random.randint(100000, 999999), 
                                   np.random.randint(10000, 99999), 
                                   np.random.randint(100000, 999999),  
                                   r_date, 
                                   int(str(datetime.today())[:4]) - int(r_date[6:])]



In [20]:
restaurant_df

,id,name,cid,sid,rid,founded,age
0,563601,gimpy-amber-cassowary,187033,88990,995927,11/28/2014,5
1,908784,dorky-brass-coati,131585,34484,688389,08/17/2017,2
2,324071,cloudy-eggplant-snake,359855,79550,681370,03/03/2018,1
3,499082,sleepy-magnolia-donkey,684467,62040,173253,08/07/2018,1
4,744153,leaky-magenta-civet,186919,26386,670133,02/05/2010,9
5,129283,ready-grey-mongrel,560628,75248,216437,08/24/2019,0
6,645445,dorky-gamboge-numbat,972416,50125,799355,05/26/2015,4
7,106082,frumpy-wisteria-crane,611975,86857,195226,02/21/2014,5
8,488204,baggy-rose-bear,956790,87929,541221,12/14/2016,3
9,200706,hazy-gamboge-lemming,785976,89891,922106,06/16/2016,3


In [35]:
coordinates = pd.DataFrame(columns = ['id', 'latitude', 'longitude', 'city', 'country', 'continent'])

coordinates['id'] = restaurant_df['cid']
df_ll = get_latitude_longititude()[:100]
coordinates['latitude'] = df_ll['Latitude']
coordinates['longitude'] = df_ll['Longitude']
coordinates['city'] = df_ll['City']
coordinates['country'] = df_ll['Country']

coordinates

,id,latitude,longitude,city,country,continent
0,187033,42.483333,1.466667,aixas,ad,NaN
1,131585,42.466667,1.500000,aixirivali,ad,NaN
2,359855,42.500000,1.516667,andorra,ad,NaN
3,684467,42.566667,1.516667,ansalonga,ad,NaN
4,186919,42.533333,1.533333,anyos,ad,NaN
5,560628,42.583333,1.516667,arans,ad,NaN
6,972416,42.450000,1.500000,aubinya,ad,NaN
7,611975,42.550000,1.516667,el pui,ad,NaN
8,956790,42.616667,1.550000,el serrat,ad,NaN
9,785976,42.516667,1.550000,engordany,ad,NaN


In [ ]:
speciality = pd.DataFrame(columns = ['id', 'calories', 'price', 'name'])
review = pd.DataFrame(columns= ['id', 'name', 'rating'])